In [1]:
import os

import dask_geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
main_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC_data/NHD_prepped_buffered'

In [3]:
huc2s = os.listdir(main_path)
huc2s.sort()

### Read in buffered flowlines

In [4]:
width = 'max'

In [5]:
data_path = os.path.join(main_path, huc2s[0])

In [6]:
file_paths = []

for file in os.listdir(data_path):
    if width in file:
        file_paths.append(os.path.join(data_path, file))

In [7]:
file_paths[0]

'/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC_data/NHD_prepped_buffered/HUC2_01/NHDPLUS_H_0105_HU4_GDB_prepped_buffered_max.parquet'

In [8]:
huc4s = [x[94:98] for x in file_paths]
huc4s.sort()

In [9]:
bieger_area = dask_geopandas.read_parquet(path=file_paths, columns=['NHDPlusID', 'buffers'])

In [10]:
bieger_area = bieger_area.compute()

In [11]:
bieger_area['area'] = bieger_area['buffers'].area

### Read in NHDArea

In [12]:
data_path = '/nas/cee-water/cjgleason/data/NHDPlus_HR'

In [13]:
file_paths = []
two = huc2s[0]

for four in huc4s:
    huc4 = 'NHDPLUS_H_' + four + '_HU4_GDB'
    
    file_path = os.path.join(data_path, two, huc4, huc4 + '.gdb')
    
    file_paths.append(file_path)

In [14]:
d = []

for file_path in file_paths:
    area = gpd.read_file(filename=file_path, layer='NHDArea', engine='pyogrio')
    d.append(area)

nhd_area = pd.concat(d)

/work/pi_cjgleason_umass_edu/.conda/envs/narrowPIXC/lib/python3.12/site-packages/pyogrio/raw.py:196: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [15]:
nhd_area = nhd_area.to_crs(epsg=3857)

In [17]:
# deer = nhd_area.clip((-8086867.086892263, 5209725.080959584, -8079861.909633098, 5218570.366543769))
# deer.plot(cmap='hsv')

In [18]:
nhd_area['area'] = nhd_area['geometry'].area

In [19]:
nhd_area['traces'] = nhd_area['geometry']

In [20]:
both = gpd.sjoin(left_df=bieger_area, right_df=nhd_area, how='inner', predicate='intersects')

In [21]:
both.columns

Index(['NHDPlusID_left', 'buffers', 'area_left', 'index_right',
       'Permanent_Identifier', 'FDate', 'Resolution', 'GNIS_ID', 'GNIS_Name',
       'AreaSqKm', 'Elevation', 'FType', 'FCode', 'VisibilityFilter',
       'Shape_Length', 'Shape_Area', 'NHDPlusID_right', 'VPUID', 'area_right',
       'traces'],
      dtype='object')

In [22]:
both.iloc[10].area_left / both.iloc[10].area_right

np.float64(0.17427788137958425)

In [ ]:
save_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/figures/figures_for_2025_04_01'

In [ ]:
both = both.to_crs(epsg=32618)

In [ ]:
deer = both.clip((684000, 4718500, 687000, 4722500))

In [ ]:
fig, ax = plt.subplots()
deer.traces.plot(ax=ax, cmap='hsv')
deer.plot(ax=ax, color='white', alpha=0.9)
# plt.xlim(684000, 687000)
# plt.ylim(4718500, 4722500)

In [ ]:
for i in range(1000):

    fig, ax = plt.subplots()
    both.iloc[[i]].traces.plot(ax=ax, color='cornflowerblue')
    both.iloc[[i]].plot(ax=ax, color='red', alpha=0.9)

    bounds = both.iloc[[i]].total_bounds
    plt.xlim(bounds[0], bounds[2]),
    plt.ylim(bounds[1], bounds[3])

    plt.savefig(os.path.join(save_path, 'bieger_nhd_area_' + str(i) + '.png'))

In [ ]:
both.iloc[[0]].traces.plot()

In [ ]:
bieger_area.plot()